In [2]:
# Import Packages

import pandas as pd
import numpy as np

In [3]:
df = pd.read_sas("2017-2020 data\P_HEQ.XPT") #testing changes

df

,SEQN,HEQ010,HEQ020,HEQ030,HEQ040
0,109264.0,2.0,NaN,2.0,NaN
1,109266.0,2.0,NaN,2.0,NaN
2,109267.0,2.0,NaN,2.0,NaN
3,109268.0,2.0,NaN,2.0,NaN
4,109270.0,2.0,NaN,2.0,NaN
...,...,...,...,...,...
13213,124817.0,2.0,NaN,2.0,NaN
13214,124818.0,2.0,NaN,2.0,NaN
13215,124820.0,2.0,NaN,2.0,NaN
13216,124821.0,2.0,NaN,2.0,NaN


In [4]:
# 2013-2014 data

Hepatitis1 = pd.read_sas("2013-2014 data\HEQ_H.XPT")
AlcoholUse1 = pd.read_sas("2013-2014 data\ALQ_H.XPT")
CurrentHealthStatus1 = pd.read_sas("2013-2014 data\HSQ_H.XPT")
Immunization1 = pd.read_sas("2013-2014 data\IMQ_H.XPT")
MedicalConditions1 = pd.read_sas("2013-2014 data\MCQ_H.XPT")
SmokingCigaretteUse1 = pd.read_sas("2013-2014 data\SMQ_H.XPT")

In [5]:
from functools import reduce
data_merge1 = reduce(lambda left, right:     # Merge three pandas DataFrames
                     pd.merge(left , right,
                              on = ["SEQN"]),
                     [AlcoholUse1, CurrentHealthStatus1,  Hepatitis1,Immunization1,MedicalConditions1,SmokingCigaretteUse1])
data_merge1

,SEQN,ALQ101,ALQ110,ALQ120Q,ALQ120U,ALQ130,ALQ141Q,ALQ141U,ALQ151,ALQ160,...,SMQ661,SMQ665A,SMQ665B,SMQ665C,SMQ665D,SMQ670,SMQ848,SMQ852Q,SMQ852U,SMAQUEX2
0,73557.0,1.0,NaN,1.000000e+00,3.0,1.0,5.397605e-79,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,73558.0,1.0,NaN,7.000000e+00,1.0,4.0,2.000000e+00,1.0,1.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0
2,73559.0,1.0,NaN,5.397605e-79,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,73561.0,1.0,NaN,5.397605e-79,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,73562.0,1.0,NaN,5.000000e+00,3.0,1.0,5.397605e-79,NaN,2.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5919,83723.0,1.0,NaN,3.000000e+00,3.0,2.0,5.397605e-79,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5920,83724.0,1.0,NaN,5.397605e-79,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5921,83726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5922,83727.0,1.0,NaN,1.000000e+00,2.0,3.0,5.397605e-79,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [6]:
data_merge1.shape

(5924, 154)

In [7]:
# 2015-2016 data

Hepatitis2 = pd.read_sas("2015-2016 data\HEQ_I.XPT")
AlcoholUse2 = pd.read_sas("2015-2016 data\ALQ_I.XPT")
CurrentHealthStatus2 = pd.read_sas("2015-2016 data\HSQ_I.XPT")
Immunization2 = pd.read_sas("2015-2016 data\IMQ_I.XPT")
MedicalConditions2 = pd.read_sas("2015-2016 data\MCQ_I.XPT")
SmokingCigaretteUse2 = pd.read_sas("2015-2016 data\SMQ_I.XPT")

In [8]:
from functools import reduce
data_merge2 = reduce(lambda left, right:     # Merge three pandas DataFrames
                     pd.merge(left , right,
                              on = ["SEQN"]),
                     [AlcoholUse2, CurrentHealthStatus2,  Hepatitis2,Immunization2,MedicalConditions2,SmokingCigaretteUse2])
data_merge2

,SEQN,ALQ101,ALQ110,ALQ120Q,ALQ120U,ALQ130,ALQ141Q,ALQ141U,ALQ151,ALQ160,...,SMQ930,SMQ935,SMQ080,SMQ890,SMQ895,SMQ900,SMQ905,SMQ910,SMQ915,SMAQUEX2
0,83732.0,1.0,NaN,1.000000e+00,2.0,1.0,5.397605e-79,NaN,2.0,NaN,...,NaN,NaN,NaN,2.0,NaN,2.0,NaN,1.0,5.397605e-79,1.0
1,83733.0,1.0,NaN,7.000000e+00,1.0,6.0,7.000000e+00,1.0,1.0,5.397605e-79,...,NaN,NaN,NaN,1.0,3.000000e+01,2.0,NaN,2.0,NaN,1.0
2,83734.0,1.0,NaN,5.397605e-79,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,1.0,5.397605e-79,2.0,NaN,1.0,5.397605e-79,1.0
3,83735.0,2.0,1.0,3.000000e+00,3.0,1.0,5.397605e-79,NaN,2.0,NaN,...,7.0,3.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,1.0
4,83736.0,2.0,1.0,1.000000e+00,3.0,1.0,5.397605e-79,NaN,2.0,NaN,...,NaN,NaN,NaN,1.0,7.000000e+00,2.0,NaN,2.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5730,93695.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,2.0,NaN,2.0,NaN,1.0
5731,93696.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,2.0,NaN,2.0,NaN,1.0
5732,93697.0,1.0,NaN,2.000000e+00,1.0,1.0,5.397605e-79,NaN,2.0,NaN,...,NaN,NaN,NaN,1.0,5.397605e-79,2.0,NaN,2.0,NaN,1.0
5733,93700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,2.0,NaN,2.0,NaN,1.0


In [9]:
data_merge2.shape

(5735, 159)

In [10]:
# 2017-2020 data

Hepatitis3 = pd.read_sas("2017-2020 data\P_HEQ.XPT")
AlcoholUse3 = pd.read_sas("2017-2020 data\P_ALQ.XPT")
CurrentHealthStatus3 = pd.read_sas("2017-2020 data\P_HSQ.XPT")
Immunization3 = pd.read_sas("2017-2020 data\P_IMQ.XPT")
MedicalConditions3 = pd.read_sas("2017-2020 data\P_MCQ.XPT")
SmokingCigaretteUse3 = pd.read_sas("2017-2020 data\P_SMQ.XPT")

In [11]:
from functools import reduce
data_merge3 = reduce(lambda left, right:     # Merge three pandas DataFrames
                     pd.merge(left , right,
                              on = ["SEQN"]),
                     [AlcoholUse3, CurrentHealthStatus3,  Hepatitis3,Immunization3,MedicalConditions3,SmokingCigaretteUse3])
data_merge3

,SEQN,ALQ111,ALQ121,ALQ130,ALQ142,ALQ270,ALQ280,ALQ290,ALQ151,ALQ170,...,SMD057,SMQ078,SMD641,SMD650,SMD100FL,SMD100MN,SMQ670,SMQ621,SMD630,SMAQUEX2
0,109266.0,1.0,1.000000e+01,1.0,5.397605e-79,NaN,NaN,NaN,2.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,109271.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,2.0,30.0,20.0,NaN,NaN,1.0,NaN,NaN,1.0
2,109273.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,1.0,30.0,15.0,1.0,1.0,1.0,NaN,NaN,1.0
3,109274.0,1.0,4.000000e+00,2.0,5.000000e+00,7.000000e+00,5.397605e-79,NaN,2.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,109282.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8960,124815.0,1.0,3.000000e+00,1.0,5.397605e-79,NaN,NaN,NaN,2.0,5.397605e-79,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8961,124817.0,1.0,3.000000e+00,2.0,5.397605e-79,NaN,NaN,NaN,2.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8962,124818.0,1.0,9.000000e+00,2.0,5.397605e-79,NaN,NaN,NaN,2.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8963,124821.0,1.0,5.000000e+00,5.0,7.000000e+00,5.397605e-79,5.397605e-79,NaN,2.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [12]:
data_merge3.shape

(8965, 102)

In [13]:
# Find columns that are similar across all year periods

similar_columns = sorted(list(set(data_merge1.columns).intersection(set(data_merge2.columns)).intersection(set(data_merge3))))

In [14]:
# Use columns similar across all year periods

data_20132014 = data_merge1[similar_columns]
data_20132014.insert(0, 'SEQN', data_20132014.pop('SEQN'))

data_20152016 = data_merge2[similar_columns]
data_20152016.insert(0, 'SEQN', data_20152016.pop('SEQN'))

data_20172020 = data_merge3[similar_columns]
data_20172020.insert(0, 'SEQN', data_20172020.pop('SEQN'))

In [15]:
# Concatenate the datasets

data_total = pd.concat([data_20132014, data_20152016, data_20172020])

data_total

,SEQN,AGQ030,ALQ130,ALQ151,HEQ010,HEQ020,HEQ030,HEQ040,HSQ590,IMQ011,...,SMD630,SMD641,SMD650,SMQ020,SMQ040,SMQ050Q,SMQ050U,SMQ078,SMQ621,SMQ670
0,73557.0,NaN,1.0,1.0,2.0,NaN,2.0,NaN,2.0,3.0,...,NaN,NaN,NaN,1.0,3.0,3.0,4.0,NaN,NaN,NaN
1,73558.0,2.0,4.0,1.0,2.0,NaN,2.0,NaN,2.0,3.0,...,NaN,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,2.0
2,73559.0,NaN,NaN,2.0,2.0,NaN,2.0,NaN,2.0,3.0,...,NaN,NaN,NaN,1.0,3.0,40.0,4.0,NaN,NaN,NaN
3,73561.0,NaN,NaN,2.0,2.0,NaN,2.0,NaN,2.0,3.0,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,73562.0,NaN,1.0,2.0,2.0,NaN,2.0,NaN,2.0,9.0,...,NaN,NaN,NaN,1.0,3.0,16.0,4.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8960,124815.0,NaN,1.0,2.0,2.0,NaN,2.0,NaN,1.0,3.0,...,NaN,NaN,NaN,1.0,3.0,2.0,4.0,NaN,NaN,NaN
8961,124817.0,NaN,2.0,2.0,2.0,NaN,2.0,NaN,2.0,3.0,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
8962,124818.0,NaN,2.0,2.0,2.0,NaN,2.0,NaN,1.0,1.0,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
8963,124821.0,2.0,5.0,2.0,2.0,NaN,2.0,NaN,2.0,3.0,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# People that have either hepatitis B or hepatitis 

data_total.loc[(data_total['HEQ010'] == 1.0) | (data_total['HEQ030'] == 1.0)]

,SEQN,AGQ030,ALQ130,ALQ151,HEQ010,HEQ020,HEQ030,HEQ040,HSQ590,IMQ011,...,SMD630,SMD641,SMD650,SMQ020,SMQ040,SMQ050Q,SMQ050U,SMQ078,SMQ621,SMQ670
26,73604.0,NaN,2.0,1.0,2.0,NaN,1.0,1.0,1.0,3.0,...,NaN,10.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,2.0
50,73643.0,NaN,1.0,2.0,1.0,1.0,2.0,NaN,1.0,3.0,...,NaN,NaN,NaN,1.0,3.0,5.0,4.0,NaN,NaN,NaN
133,73778.0,2.0,NaN,1.0,1.0,9.0,2.0,NaN,2.0,3.0,...,NaN,NaN,NaN,1.0,3.0,36.0,4.0,NaN,NaN,NaN
137,73782.0,NaN,1.0,2.0,2.0,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,1.0,3.0,14.0,4.0,NaN,NaN,NaN
171,73843.0,NaN,2.0,2.0,1.0,2.0,2.0,NaN,1.0,1.0,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8660,124315.0,2.0,NaN,NaN,2.0,NaN,1.0,2.0,NaN,3.0,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
8728,124419.0,1.0,5.0,1.0,2.0,NaN,1.0,2.0,1.0,3.0,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
8797,124553.0,1.0,NaN,NaN,2.0,NaN,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
8922,124757.0,NaN,1.0,2.0,1.0,2.0,2.0,NaN,1.0,9.0,...,NaN,NaN,NaN,1.0,3.0,12.0,4.0,NaN,NaN,NaN


In [17]:
# HEQ010 1.0 or HEQ030 1.0 (has hepatitis) new column

data_total['HEQ'] = np.where(((data_total.HEQ010 == 1.0) | ((data_total.HEQ030 == 1.0))), 1.0, 2.0)

data_total['HEQ'].value_counts()

2.0    20098
1.0      526
Name: HEQ, dtype: int64

In [18]:
# HEQ020 1.0 or HEQ040 1.0 (took meds for hepatitis) new column

data_total['MEDS'] = np.where(((data_total.HEQ020 == 1.0) | ((data_total.HEQ040 == 1.0))), 1.0, 2.0)

data_total['MEDS'].value_counts()

2.0    20452
1.0      172
Name: MEDS, dtype: int64

In [19]:
# Drop HEQ columns as we have combined them

data_total = data_total.drop(['HEQ010', 'HEQ020', 'HEQ030', 'HEQ040'], axis = 1)

In [20]:
data_total

,SEQN,AGQ030,ALQ130,ALQ151,HSQ590,IMQ011,IMQ020,IMQ070,IMQ090,MCD093,...,SMD650,SMQ020,SMQ040,SMQ050Q,SMQ050U,SMQ078,SMQ621,SMQ670,HEQ,MEDS
0,73557.0,NaN,1.0,1.0,2.0,3.0,3.0,NaN,NaN,NaN,...,NaN,1.0,3.0,3.0,4.0,NaN,NaN,NaN,2.0,2.0
1,73558.0,2.0,4.0,1.0,2.0,3.0,3.0,2.0,NaN,NaN,...,1.0,1.0,2.0,NaN,NaN,NaN,NaN,2.0,2.0,2.0
2,73559.0,NaN,NaN,2.0,2.0,3.0,3.0,NaN,NaN,2.0,...,NaN,1.0,3.0,40.0,4.0,NaN,NaN,NaN,2.0,2.0
3,73561.0,NaN,NaN,2.0,2.0,3.0,3.0,NaN,NaN,1.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0
4,73562.0,NaN,1.0,2.0,2.0,9.0,9.0,2.0,NaN,3.0,...,NaN,1.0,3.0,16.0,4.0,NaN,NaN,NaN,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8960,124815.0,NaN,1.0,2.0,1.0,3.0,3.0,2.0,NaN,NaN,...,NaN,1.0,3.0,2.0,4.0,NaN,NaN,NaN,2.0,2.0
8961,124817.0,NaN,2.0,2.0,2.0,3.0,3.0,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0
8962,124818.0,NaN,2.0,2.0,1.0,1.0,3.0,2.0,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0
8963,124821.0,2.0,5.0,2.0,2.0,3.0,3.0,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0


In [21]:
print(data_total.columns)

Index(['SEQN', 'AGQ030', 'ALQ130', 'ALQ151', 'HSQ590', 'IMQ011', 'IMQ020',
       'IMQ070', 'IMQ090', 'MCD093', 'MCQ010', 'MCQ025', 'MCQ035', 'MCQ040',
       'MCQ050', 'MCQ053', 'MCQ080', 'MCQ092', 'MCQ149', 'MCQ151', 'MCQ160A',
       'MCQ160B', 'MCQ160C', 'MCQ160D', 'MCQ160E', 'MCQ160F', 'MCQ160L',
       'MCQ160M', 'MCQ170L', 'MCQ170M', 'MCQ195', 'MCQ220', 'MCQ230A',
       'MCQ230B', 'MCQ230C', 'MCQ230D', 'MCQ300A', 'MCQ300B', 'MCQ300C',
       'SMAQUEX2', 'SMD030', 'SMD057', 'SMD100FL', 'SMD100MN', 'SMD630',
       'SMD641', 'SMD650', 'SMQ020', 'SMQ040', 'SMQ050Q', 'SMQ050U', 'SMQ078',
       'SMQ621', 'SMQ670', 'HEQ', 'MEDS'],
      dtype='object')


,number_missing,percentage_missing
MCQ149,20624,100.000000
SMQ621,20624,100.000000
SMD630,20624,100.000000
MCQ151,20624,100.000000
MCQ230D,20619,99.975756
MCQ230C,20596,99.864236
MCQ230B,20410,98.962374
MCQ170L,19710,95.568270
IMQ090,18964,91.951125
MCQ050,18725,90.792281


SEQN        0
IMQ011      0
IMQ020      0
MCQ010      0
MCQ053      0
MCQ080      0
MCQ092      0
MCQ300B     0
SMAQUEX2    0
SMQ020      0
HEQ         0
MEDS        0
dtype: int64